In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import pytorch_lightning as pl
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import pandas as pd

train_dataset_name = 'ArithOpsTrain.xlsx'
df = pd.read_excel(train_dataset_name)
df = df.drop('Table 1',axis=1)
df = df.rename(columns=df.iloc[0]).loc[1:]

train_df , valid_df = train_test_split(df,test_size=0.1,random_state=0)

In [3]:
class T5Dataset(Dataset):
    def __init__(
        self,
        data  : pd.DataFrame,
        tokenizer : T5Tokenizer,
        text_max_token_length = 512,
        output_max_token_length = 128
    ):
        
        super().__init__()
        self.tokenizer = tokenizer
        self.data = data 
        self.text_max_token_length = text_max_token_length,
        self.output_max_token_length = output_max_token_length
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        
        data_row = self.data.iloc[index]

        input_text = data_row["Description"]
        input_text_encoding = self.tokenizer(
            input_text,
            max_length=100,
            padding = "max_length",
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
        )

        output_text = data_row["Equation"]        
        output_text_encoding = self.tokenizer(
            output_text,
            max_length=100,
            padding = "max_length",
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
        )

        return dict(
            input_text = input_text,
            output_text = output_text,
            input_text_ids = input_text_encoding['input_ids'].flatten(),
            input_attention_mask = input_text_encoding['attention_mask'].flatten(),
            output_text_ids = output_text_encoding['input_ids'].flatten(),
            output_attention_mask = output_text_encoding['attention_mask'].flatten()
        )

        

In [4]:
from re import S
from torch.utils.data import DataLoader

class T5DataSetModule(pl.LightningDataModule):

    def __init__(
        self,
        train_df : pd.DataFrame,
        valid_df : pd.DataFrame,
        tokenizer : T5Tokenizer,
        batch_size = 64,
        text_max_token_length = 512,
        output_max_token_length = 128
    ):
        super().__init__()
        self.train_df = train_df
        self.valid_df = valid_df
        self.tokenizer = tokenizer,
        self.batch_size = batch_size,
        self.text_max_token_length = text_max_token_length
        self.output_max_token_length = output_max_token_length

    
    def setup(self, stage = None):
        
        self.train_dataset = T5Dataset(self.train_df,self.tokenizer,self.text_max_token_length,self.output_max_token_length)
        self.valid_dataset = T5Dataset(self.valid_df,self.tokenizer,self.text_max_token_length,self.output_max_token_length)
    
    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            self.batch_size,
            shuffle=True,
        )
    
    def val_dataloader(self):
        return DataLoader(
            self.valid_dataset,
            self.batch_size,
            shuffle=False
        )
        

In [5]:
t5_tokenizer = T5Tokenizer.from_pretrained("t5-small")
t5_model =T5ForConditionalGeneration.from_pretrained("t5-small")

/Users/depressedcoder/environments/gymenv/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5.py:164: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [6]:
N_EPOCHS = 3
BATCH_SIZE = 32

data_module = T5DataSetModule(train_df,valid_df,t5_tokenizer,BATCH_SIZE,)

In [ ]:
def postfix_evaluation(batch_data,input_values):

    arith_symbols = set(['+','-','*','/','%'])
    output_values = []
    
    for i in range(len(batch_data)):
        flag = True
        current_input = batch_data[i].split(' ')
        current_input.reverse()
        input_value = input_values[i]

        stack = []
        for symbol in current_input:
            if symbol in arith_symbols:
                if len(stack)<2:
                    flag = False
                    break
                in1 = stack.pop(-1)
                in2 = stack.pop(-1)

                res = 0
                if symbol=='+':
                    res = in1+in2
                elif symbol=='-':
                    res = in1 - in2 
                elif symbol == '*':
                    res = in1 * in2
                elif symbol=='/':
                    res = in1/in2
                else:
                    res = in1 % in2
                stack.append(res)


            else:
                if "number" in symbol:
                    index = int(symbol[6])
                    stack.append(input_value[index])

        if flag==False or len(stack)!=1:
            output_values.append(0)
        else:
            output_values.append(stack.pop(-1))
    
    return output_values

ans = postfix_evaluation(["+ - number0 number1 number2","+ / - number0 number2 number1 number3"],[[1,4,6],[5,6,7,8]])

In [ ]:
class T5ArithTranslator(pl.LightningModule):

    def __init__(self):
        super().__init__()